In [ ]:
# 1. Forced Install (Silent Mode)
print("🏗️ Installing Engine... (Please wait)")
!apt-get update -qq > /dev/null
!apt-get install -y docker.io nodejs npm adb -qq > /dev/null

# 2. Launch Android Engine
print("📱 Starting Android... (30s)")
!docker run -d --privileged -p 5555:5555 redroid/redroid:11.0.0-latest > /dev/null

# 3. Clone & Build Viewer (Direct Method)
print("🌐 Setting up Web Interface...")
!git clone https://github.com/NetrisTV/ws-scrcpy.git -q
%cd ws-scrcpy
!npm install --quiet > /dev/null
!npm run dist > /dev/null

# 4. Tunnel & Launch
print("🔗 GENERATING YOUR LINK BELOW:")
import subprocess
import threading

def run_app():
    subprocess.run(["npm", "start", "--", "--port", "7860"], stdout=subprocess.DEVNULL)

threading.Thread(target=run_app, daemon=True).start()

!npx localtunnel --port 7860